In [ ]:
import io
import os
from urllib import parse
from xml import etree
import zipfile

import requests

import numpy as np

from scipy import spatial

from astropy import units as u
from astropy import table
from astropy import wcs
from astropy.coordinates import Distance

from astropy.io import fits

In [ ]:
import glue_jupyter as gj
from glue import core as gcore

# Load the Pleiades 

We start by downloading the Gaia DR2 dataset for the 3 degree area around the Pleiades using `astroquery`

In [ ]:
from astroquery import gaia

The below will either download the dataset and save it locally, or it will load the saved copy if you've already downloaded.

In [ ]:
tabfn = 'M45_gaia.ecsv'
if os.path.isfile(tabfn):
    tab = table.Table.read(tabfn)
else:
    tab = gaia.Gaia.query_object_async('Messier 45', 90*u.arcmin)
    tab.write(tabfn, format='ascii.ecsv')
tab = table.QTable(tab)
len(tab)

Now lets select a sub-set of interesting columns, and sub-sample on the part of the dataset that has a valid distance and is within a reasonable range that could be anywhere near us.

In [ ]:
subtab = tab['designation', 'ra', 'dec', 'parallax', 'parallax_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 
             'phot_g_mean_mag' ,'bp_rp']

subtab['distance'] = u.Quantity(subtab['parallax']).to(u.pc, u.parallax())
subtab['distance_error'] = u.Quantity(subtab['parallax_error']).to(u.pc, u.parallax())

subtab = subtab[np.isfinite(subtab['distance'])&(u.Quantity(subtab['distance'])<300*u.pc)&(subtab['distance']>0)]
len(subtab)

# Set up Glupyter 

Now we set up a data object for Glupyter, and create a Glupyter app for further examination.

In [ ]:
len(subtab.colnames)

In [ ]:
gaia_data = gcore.Data('Gaia_Pleiades', **subtab)

In [ ]:
#a million points... 
#t2=table.vstack([subtab]*400)
#gaia_data = gcore.Data('Gaia_Pleiades', **t2)

In [ ]:
app = gj.jglue(gaia_data)

In [ ]:
app.scatter3d('ra', 'dec', 'distance')

In [ ]:
app.scatter2d('distance', 'phot_g_mean_mag')

In [ ]:
cmd = app.scatter2d('bp_rp', 'phot_g_mean_mag')
cmd.state.y_max, cmd.state.y_min = cmd.state.y_min, cmd.state.y_max

In [ ]:
scatpm = app.scatter2d('pmra', 'pmdec')
scatpm.scale_x.min = -100
scatpm.scale_x.max = 100
scatpm.scale_y.min = -100
scatpm.scale_y.max = 100

In [ ]:
scatra = app.scatter2d('pmra', 'pmra_error')
scatra.state.x_min, scatra.state.x_max = -50, 50

scatdec = app.scatter2d('pmdec', 'pmdec_error')
scatdec.state.x_min, scatdec.state.x_max = -100, 50

Now go ahead a select a subset using all the plots above...

In [ ]:
seltab = subtab[gaia_data.subsets[0].to_mask()]

And compute the distance to the Pleiades!

In [ ]:
np.mean(seltab['distance'])

Or some percentiles

In [ ]:
np.percentile(seltab['distance'], [10, 32, 50, 68, 90])

And while we're at it - lets see what the mean proper motion is:

In [ ]:
np.mean(seltab['pmra']), np.mean(seltab['pmdec'])

In [ ]:
np.mean(np.hypot(seltab['pmra']*np.cos(seltab['dec']), seltab['pmdec']))

# Now overlay HST image 

Now we look at an HST image of a small part of the Pleiades, and try overplotting some of the Gaia stars

In [ ]:
from astroquery.mast import Observations

In [ ]:
mast_qry = Observations.query_criteria(target_name='PLEIADESFIELD1B', obs_collection='hst')
mast_qry

Now lets download just the final drizzled image for the observation with the bluer filter (highest resolution): F475W

In [ ]:
result = Observations.download_products(mast_qry['obsid'][mast_qry['filters']=='F475W'], 
                                        productSubGroupDescription='DRZ')
result['Local Path']

In [ ]:
assert len(result) == 1

f475w_image = fits.open(result['Local Path'][0])

Now we add the image data to glue and display it:

In [ ]:
sciim = app.add_data(f475w=f475w_image)[0]
imview = app.imshow(data=sciim)

Now we glue together the image and the Gaia data. (Note that it might take some time for the view to catch up after you make the link.)

In [ ]:
app.add_link(gaia_data, 'ra', sciim, 'Right Ascension')
app.add_link(gaia_data,'dec', sciim, 'Declination')

imview.add_data(gaia_data)

Markers should appear in the image viewer. You may need to fiddle with the color a bit to see the markers.  Since you've already made your subset selection: are any of the Pleiades members in the HST field?

You'll probably see them: they are there, but clearly offset from where the HST image has them!  This might at first appear to be a bug, but it's not.  Why might this be?

# Get Isochrone 

In [ ]:
data = {'version': 1.2,
'v_div_vcrit': 'vvcrit0.4',
'age_scale': 'log10',
'age_value': 6,
'age_type': 'range',
'age_range_low': 5.01,
'age_range_high': 8.01,
'age_range_delta': .2,
'age_list': '',
'FeH_value': 0.03,
'theory_output': 'basic',
'output_option': 'photometry',
'output': 'UBVRIplus',
'Av_value': 0}

isoc_search = requests.post('http://waps.cfa.harvard.edu/MIST/iso_form.php', data)
isoc_search

In [ ]:
et = etree.ElementTree.fromstring(isoc_search.text)
relative_path = et.find('.//a').attrib['href']
absolute_url = parse.urljoin(isoc_search.url, relative_path)
absolute_url

In [ ]:
isoc_download = requests.get(absolute_url)

zip_file = zipfile.ZipFile(io.BytesIO(isoc_download.content))
isoc_file = zip_file.open(zip_file.namelist()[0])
isoc = table.Table.read(isoc_file, format='ascii.commented_header', 
                        header_start=12, guess=False)

In [ ]:
iG = isoc['Gaia_G_DR2Rev']
ibprp = isoc['Gaia_BP_DR2Rev'] - isoc['Gaia_RP_DR2Rev']

In [ ]:
#or to reload the downloaded version:
#isoc = table.Table.read('MIST_iso_5b89469f74d5e.iso.cmd', format='ascii.commented_header',header_start=12, guess=False)

## Experiment w/ isochrone 

Now we try adding an isochrone of stars over the top of the above.  Note that this is *completely* outside the Glue ecosystem, and would be exceedingly difficult to shoehorn into glue without custom file-writing.

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt

In [ ]:
misoc = isoc[isoc['log10_isochrone_age_yr']==7.41]
miG = misoc['Gaia_G_DR2Rev']
mibprp = misoc['Gaia_BP_DR2Rev'] - misoc['Gaia_RP_DR2Rev']

In [ ]:
kdt = spatial.cKDTree(np.array([mibprp, miG + Distance(140*u.pc).distmod.value]).T)
d, idx = kdt.query(np.array([gaia_data['bp_rp'], gaia_data['phot_g_mean_mag']]).T)

In [ ]:
plt.figure(figsize=(12, 12))

msk = d<.2
plt.scatter(gaia_data['bp_rp'][msk], gaia_data['phot_g_mean_mag'][msk], c=d[msk], vmin=0, vmax=.5)
plt.scatter(gaia_data['bp_rp'][~msk], gaia_data['phot_g_mean_mag'][~msk], c=d[~msk], vmin=0, vmax=.5, edgecolors='r')
plt.colorbar()


plt.ylim(20,-2)

## Apply subset 

In [ ]:
gaia_data.add_component(d, 'cmd_distance')

In [ ]:
app.data_collection.new_subset_group('isoc2', gaia_data.id['cmd_distance']<.2)

In [ ]:
app.data_collection[0].subsets[1].style.color = 'blue'

# How does this act like a GUI? 

In [ ]:
import ipywidgets

In [ ]:
app2 = gj.jglue(gcore.Data('Gaia_Pleiades', **subtab))
scatpm = app2.scatter2d('pmra', 'pmdec')
for s in (scatpm.scale_x, scatpm.scale_y):
    s.min = -100
    s.max = 100
cmd = app2.scatter2d('bp_rp', 'phot_g_mean_mag')
cmd.state.y_max, cmd.state.y_min = cmd.state.y_min, cmd.state.y_max

In [ ]:
b1 = ipywidgets.Button(description='Select Cluster')
def on_b1_clicked(b):
    data = app2.data_collection[0]
    lowerror = (data.id['pmra_error'] < 1) & (data.id['pmdec_error'] < 1)
    in_cluster = (data.id['pmra'] - 20)**2 + (data.id['pmdec'] - -45)**2 < (10)**2
    sub = app2.data_collection.new_subset_group('cluster_circle', lowerror & in_cluster)
    
    
b1.on_click(on_b1_clicked)

In [ ]:
b2 = ipywidgets.Button(description='Select Milky Way')
def on_b2_clicked(b):
    data = app2.data_collection[0]
    lowerror = (data.id['pmra_error'] < 1) & (data.id['pmdec_error'] < 1)
    in_cluster = (data.id['pmra'])**2 + (data.id['pmdec'])**2 < (30)**2
    sub = app2.data_collection.new_subset_group('mw_circle', lowerror & in_cluster)
    
    
b2.on_click(on_b2_clicked)

In [ ]:
mainv = ipywidgets.VBox([ipywidgets.HBox([b1, b2]), 
                         ipywidgets.HBox([scatpm.figure, cmd.figure])])
mainv